In [1]:
import os
import pyodbc
import numpy as np
import pandas as pd
from datetime import datetime

## Step 2. Connect to LASR

In [65]:
cnxn = pyodbc.connect('Driver={SQL Server Native Client 11.0};\
                       Server=lasr-sqldb-prd-im,17001;\
                       Database=IM_I_IDWBRRESULTS_S;\
                       Trusted_Connection=yes;')

##### RA_PORT_RTN and RA_PORT_BMRK_COMPONENT

In [66]:
sql_query = """
select
pfm.RAPORTFOLIOID,
pfm.postdt,
COMP.raportbenchmarkid,
COMP.STARTDT,
COMP.enddt,
pfm.RTNPERIODTYPCD,
cg.InvestmentPortfolioTypeCode,
ia.InvestmentAccountTypeCode,
ia.SeparateAccountTypeCode,
ia.FundAccountTypeCode,
ia.FundAccountSubTypeCode,
ia.FundAccountGroupTypeCode
from
(
select
RAPORTFOLIOID,
RTNPERIODENDDT as postdt,
RTNPERIODTYPCD,
RAMARKETCYCLEID
from IM_I_IDWBRRADM_S.dbo.RA_PORT_RTN
where CRNCYUID = 1 and RTNPCT is not NULL and RTNPERIODTYPCD IN ('1MO','3MO','6MO','12MO')
) as pfm
  inner join
(
select
RAPORTFOLIOID,
RTNPERIODENDDT as postdt,
RTNPERIODTYPCD,
max(RAMARKETCYCLEID) as max_cycle
from IM_I_IDWBRRADM_S.dbo.RA_PORT_RTN
where CRNCYUID = 1 and RTNPCT is not NULL
group by RAPORTFOLIOID,RTNPERIODENDDT,RTNPERIODTYPCD
) as B
on pfm.RAPORTFOLIOID=B.RAPORTFOLIOID and pfm.postdt=B.postdt and pfm.RTNPERIODTYPCD=B.RTNPERIODTYPCD and pfm.RAMARKETCYCLEID=B.max_cycle 

left join
(
select
VWB.RAPORTFOLIOID,
VWB.RAPORTBENCHMARKID,
VWB.STARTDT,
VWB.ENDDT
from
  (
       select
       VBMK.RAPORTFOLIOID,
       VBMK.RAPORTBENCHMARKID,
       BCMP.STARTDT,
       BCMP.ENDDT
       from IM_M_RADM_S.dbo.vwRaPortBenchmark as VBMK
         left join 
               IM_I_IDWBRRADM_S.dbo.RA_PORT_BMRK_COMPONENT as BCMP
         on VBMK.RAPORTBENCHMARKID = BCMP.RAPORTBENCHMARKID and BCMP.DELETEIND <> 'Y'
  ) as VWB
) as COMP
on pfm.RAPORTFOLIOID = COMP.RAPORTFOLIOID and pfm.postdt between COMP.STARTDT and COMP.ENDDT

left join
	(
	select
	RAPORTFOLIOID,
	PORTUID as portfoliouid,
	INVPORTID as aggrid
	from IM_I_IDWBRRADM_S.dbo.RA_PORTFOLIO
	) as XW
on pfm.RAPORTFOLIOID = XW.RAPORTFOLIOID
-- the labeling stuff here --
  left join 
     IM_I_PORTFOLIOPOS_S.dbo.cgportfolio as cg
  on XW.PortfolioUID=cg.PortfolioUID and pfm.postdt between  cg.dweffectivefromdate and cg.dweffectivetodate

  left join
     IM_U_RESULTS_S.KPMG.KMPYAD_AA as AA
  on cg.portfoliouid =  AA.AAPortfolioUID and pfm.postdt between  AA.dweffectivefromdate and AA.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AA'
  left join
     IM_U_RESULTS_S.KPMG.KMPYAD_AR as AR
  on cg.portfoliouid =  AR.ARPortfolioUID and pfm.postdt between  AR.dweffectivefromdate and AR.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AR'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_AT as AT
  on cg.portfoliouid =  AT.ATPortfolioUID and pfm.postdt between  AT.dweffectivefromdate and AT.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AT'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_AM as AM
  on cg.portfoliouid =  AM.AMPortfolioUID and pfm.postdt between  AM.dweffectivefromdate and AM.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AM'
  --the extra non-jasper ones
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_GA as GA
  on cg.portfoliouid =  GA.GAPortfolioUID and pfm.postdt between  GA.dweffectivefromdate and GA.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'GA'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_ID as ID
  on cg.portfoliouid =  ID.IDPortfolioUID and pfm.postdt between  ID.dweffectivefromdate and ID.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'ID'
  left join
     IM_I_ACCTPOSTRANS_S.dbo.InvestmentAccount as ia
	       ---case it out---
  on ia.InvestmentAccountUID = case when cg.InvestmentPortfolioTypeCode='AA' THEN AA.InvestmentAccountUID
                                    when cg.InvestmentPortfolioTypeCode='AR' THEN AR.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='AT' THEN AT.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='AM' THEN AM.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='GA' THEN GA.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='ID' THEN ID.InvestmentAccountUID
								end
  and pfm.postdt between ia.dweffectivefromdate and ia.dweffectivetodate
"""

# Save the data
port_df = pd.read_sql(sql_query, cnxn)
    
print(port_df.shape)
port_df.head()

(9718201, 12)


,RAPORTFOLIOID,postdt,raportbenchmarkid,STARTDT,enddt,RTNPERIODTYPCD,InvestmentPortfolioTypeCode,InvestmentAccountTypeCode,SeparateAccountTypeCode,FundAccountTypeCode,FundAccountSubTypeCode,FundAccountGroupTypeCode
0,1028,2004-07-30,26592.0,1982-12-31,2006-01-31 00:00:00,1MO,FL,None,None,None,None,None
1,16892,2016-12-30,11049.0,2012-02-29,9999-12-31 00:00:00,1MO,AM,FUND,None,MF,AF,NG
2,232413,2017-03-31,15623.0,2016-12-30,2018-07-31 00:00:00,6MO,AM,FUND,None,MF,AF,VI
3,152008,2009-02-27,18779.0,1993-05-28,9999-12-31 00:00:00,12MO,AT,CM,None,None,None,None
4,242155,2016-12-30,NaN,NaT,None,6MO,AM,FUND,None,MF,AF,NG


In [67]:
port_df.isnull().sum()

RAPORTFOLIOID                        0
postdt                               0
raportbenchmarkid               430958
STARTDT                         430958
enddt                           430958
RTNPERIODTYPCD                       0
InvestmentPortfolioTypeCode     929799
InvestmentAccountTypeCode      6194991
SeparateAccountTypeCode        9673337
FundAccountTypeCode            9204919
FundAccountSubTypeCode         9381991
FundAccountGroupTypeCode       9244664
dtype: int64

#### RA_BMRK_RTN

In [68]:
sql_query = """
select
VWB.raportfolioid,
VWB.raportbenchmarkid,
BMK.rtnperiodenddt as postdt,
BMK.rtnperiodtypcd
from
  (
	select
	VBMK.RAPORTFOLIOID,
	VBMK.RAPORTBENCHMARKID
	from IM_M_RADM_S.dbo.vwRaPortBenchmark as VBMK
  ) as VWB
  left join
  (
	select
	A.raportbenchmarkid,
	A.RTNPERIODENDDT,
	A.RTNPERIODTYPCD
	from IM_I_IDWBRRADM_S.dbo.RA_BMRK_RTN as A
	  inner join
		  (
		  select
		  raportbenchmarkid,
		  RTNPERIODENDDT,
		  RTNPERIODTYPCD,
		  max(RAMARKETCYCLEID) as max_RAMARKETCYCLEID
		  from IM_I_IDWBRRADM_S.dbo.RA_BMRK_RTN
		  where crncyuid = 1 and rtnpct is not null
		  group by raportbenchmarkid, RTNPERIODENDDT, RTNPERIODTYPCD
		  ) as B
	  on A.raportbenchmarkid=B.raportbenchmarkid and A.RTNPERIODENDDT=B.RTNPERIODENDDT and A.RTNPERIODTYPCD=B.RTNPERIODTYPCD and A.RAMARKETCYCLEID=B.max_RAMARKETCYCLEID
		 and A.crncyuid=1 and A.rtnpct is not NULL
  ) as BMK
on BMK.raportbenchmarkid=VWB.raportbenchmarkid
where BMK.RTNPERIODENDDT is not NULL
 and BMK.RTNPERIODTYPCD IN ('1MO','3MO','6MO','12MO')
"""



# Save the data
bmrk_df = pd.read_sql(sql_query, cnxn)
    
print(bmrk_df.shape)
bmrk_df.head()

#close connection
cnxn.close()

(9071632, 4)


## Step 3. Exploratory Data Analysis

#### RA_PORT_RTN

In [69]:
port_df.head()

,RAPORTFOLIOID,postdt,raportbenchmarkid,STARTDT,enddt,RTNPERIODTYPCD,InvestmentPortfolioTypeCode,InvestmentAccountTypeCode,SeparateAccountTypeCode,FundAccountTypeCode,FundAccountSubTypeCode,FundAccountGroupTypeCode
0,1028,2004-07-30,26592.0,1982-12-31,2006-01-31 00:00:00,1MO,FL,None,None,None,None,None
1,16892,2016-12-30,11049.0,2012-02-29,9999-12-31 00:00:00,1MO,AM,FUND,None,MF,AF,NG
2,232413,2017-03-31,15623.0,2016-12-30,2018-07-31 00:00:00,6MO,AM,FUND,None,MF,AF,VI
3,152008,2009-02-27,18779.0,1993-05-28,9999-12-31 00:00:00,12MO,AT,CM,None,None,None,None
4,242155,2016-12-30,NaN,NaT,None,6MO,AM,FUND,None,MF,AF,NG


#### RA_BMRK_RTN

In [70]:
bmrk_df.head()

,raportfolioid,raportbenchmarkid,postdt,rtnperiodtypcd
0,156615,11528,1999-11-30,6MO
1,60765,24377,2006-04-28,3MO
2,235526,11737,1984-03-30,1MO
3,250855,16087,2005-10-31,12MO
4,117725,2909,2007-11-30,6MO


## Step 4. Data Preprocessing

#### RA_PORT_RTN

In [72]:
port_df['priority'] = port_df['RTNPERIODTYPCD'].map({'1MO':1, '3MO':2, '6MO':3, '12MO':4})
port_df['postdt_new'] = port_df['postdt'].apply(lambda x: x.replace(day=1))
port_df.head()

,RAPORTFOLIOID,postdt,raportbenchmarkid,STARTDT,enddt,RTNPERIODTYPCD,InvestmentPortfolioTypeCode,InvestmentAccountTypeCode,SeparateAccountTypeCode,FundAccountTypeCode,FundAccountSubTypeCode,FundAccountGroupTypeCode,priority,postdt_new
0,1028,2004-07-30,26592.0,1982-12-31,2006-01-31 00:00:00,1MO,FL,None,None,None,None,None,1,2004-07-01
1,16892,2016-12-30,11049.0,2012-02-29,9999-12-31 00:00:00,1MO,AM,FUND,None,MF,AF,NG,1,2016-12-01
2,232413,2017-03-31,15623.0,2016-12-30,2018-07-31 00:00:00,6MO,AM,FUND,None,MF,AF,VI,3,2017-03-01
3,152008,2009-02-27,18779.0,1993-05-28,9999-12-31 00:00:00,12MO,AT,CM,None,None,None,None,4,2009-02-01
4,242155,2016-12-30,NaN,NaT,None,6MO,AM,FUND,None,MF,AF,NG,3,2016-12-01


##### Handle the NULLs in RA_PORT_RTN

In [74]:
null_benchmarkids = port_df[port_df['raportbenchmarkid'].isnull()]
null_benchmarkids.raportbenchmarkid.fillna('N/A', inplace=True)
null_benchmarkids.head()

C:\Users\KPMYAD\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,RAPORTFOLIOID,postdt,raportbenchmarkid,STARTDT,enddt,RTNPERIODTYPCD,InvestmentPortfolioTypeCode,InvestmentAccountTypeCode,SeparateAccountTypeCode,FundAccountTypeCode,FundAccountSubTypeCode,FundAccountGroupTypeCode,priority,postdt_new
4,242155,2016-12-30,N/A,NaT,None,6MO,AM,FUND,None,MF,AF,NG,3,2016-12-01
10,127266,2017-02-28,N/A,NaT,None,12MO,AA,SA,IA,None,None,None,4,2017-02-01
36,242155,2016-12-30,N/A,NaT,None,3MO,AM,FUND,None,MF,AF,NG,2,2016-12-01
42,127266,2017-01-31,N/A,NaT,None,6MO,AA,SA,IA,None,None,None,3,2017-01-01
68,242155,2016-12-30,N/A,NaT,None,1MO,AM,FUND,None,MF,AF,NG,1,2016-12-01


In [79]:
port_df.drop(['RTNPERIODTYPCD', 'STARTDT', 'enddt','postdt'], axis=1, inplace=True)
port_df.rename(columns={'RAPORTFOLIOID':'raportfolioid', 'postdt_new': 'postdt', 'InvestmentPortfolioTypeCode': 'Portype', 
                    'InvestmentAccountTypeCode': 'lab1', 'SeparateAccountTypeCode': 'lab2', 'FundAccountTypeCode': 'lab3',
                    'FundAccountSubTypeCode': 'lab4', 'FundAccountGroupTypeCode': 'lab5'}, inplace=True)
port_df.head()

,raportfolioid,raportbenchmarkid,Portype,lab1,lab2,lab3,lab4,lab5,priority,postdt
0,1028,26592.0,FL,None,None,None,None,None,1,2004-07-01
1,16892,11049.0,AM,FUND,None,MF,AF,NG,1,2016-12-01
2,232413,15623.0,AM,FUND,None,MF,AF,VI,3,2017-03-01
3,152008,18779.0,AT,CM,None,None,None,None,4,2009-02-01
5,292128,23070.0,AM,FUND,None,MF,AF,VI,1,2017-10-01


In [114]:
port_df.drop(['RTNPERIODTYPCD', 'STARTDT', 'enddt','postdt'], axis=1, inplace=True)
port_df.rename(columns={'RAPORTFOLIOID':'raportfolioid', 'postdt_new': 'postdt', 'InvestmentPortfolioTypeCode': 'Portype', 
                    'InvestmentAccountTypeCode': 'lab1', 'SeparateAccountTypeCode': 'lab2', 'FundAccountTypeCode': 'lab3',
                    'FundAccountSubTypeCode': 'lab4', 'FundAccountGroupTypeCode': 'lab5'}, inplace=True)
port_df = port_df.fillna('N/A')
port_df.head()

,raportfolioid,raportbenchmarkid,Portype,lab1,lab2,lab3,lab4,lab5,priority,postdt
0,1028,26592.0,FL,N/A,N/A,N/A,N/A,N/A,1,2004-07-01
1,16892,11049.0,AM,FUND,N/A,MF,AF,NG,1,2016-12-01
2,232413,15623.0,AM,FUND,N/A,MF,AF,VI,3,2017-03-01
3,152008,18779.0,AT,CM,N/A,N/A,N/A,N/A,4,2009-02-01
5,292128,23070.0,AM,FUND,N/A,MF,AF,VI,1,2017-10-01


In [80]:
null_benchmarkids.drop(['RTNPERIODTYPCD', 'STARTDT', 'enddt','postdt'], axis=1, inplace=True)
null_benchmarkids.rename(columns={'RAPORTFOLIOID':'raportfolioid', 'postdt_new': 'postdt', 'InvestmentPortfolioTypeCode': 'Portype', 
                    'InvestmentAccountTypeCode': 'lab1', 'SeparateAccountTypeCode': 'lab2', 'FundAccountTypeCode': 'lab3',
                    'FundAccountSubTypeCode': 'lab4', 'FundAccountGroupTypeCode': 'lab5'}, inplace=True)

null_benchmarkids_filled = null_benchmarkids.fillna('N/A')
null_benchmarkids_filled.head()

,raportfolioid,raportbenchmarkid,Portype,lab1,lab2,lab3,lab4,lab5,priority,postdt
4,242155,N/A,AM,FUND,N/A,MF,AF,NG,3,2016-12-01
10,127266,N/A,AA,SA,IA,N/A,N/A,N/A,4,2017-02-01
36,242155,N/A,AM,FUND,N/A,MF,AF,NG,2,2016-12-01
42,127266,N/A,AA,SA,IA,N/A,N/A,N/A,3,2017-01-01
68,242155,N/A,AM,FUND,N/A,MF,AF,NG,1,2016-12-01


In [115]:
#dedup
port_df.drop_duplicates(subset=['raportfolioid', 'postdt','raportbenchmarkid', 'priority'], keep='first')
print(port_df.shape)
port_df.head()

(9204802, 10)


,raportfolioid,raportbenchmarkid,Portype,lab1,lab2,lab3,lab4,lab5,priority,postdt
0,1028,26592.0,FL,N/A,N/A,N/A,N/A,N/A,1,2004-07-01
1,16892,11049.0,AM,FUND,N/A,MF,AF,NG,1,2016-12-01
2,232413,15623.0,AM,FUND,N/A,MF,AF,VI,3,2017-03-01
3,152008,18779.0,AT,CM,N/A,N/A,N/A,N/A,4,2009-02-01
5,292128,23070.0,AM,FUND,N/A,MF,AF,VI,1,2017-10-01


In [116]:
#dedup
port_df.drop_duplicates(subset=['raportfolioid', 'postdt','raportbenchmarkid', 'priority'], keep='first', inplace=True)
print(port_df.shape)
port_df.head()

(9204802, 10)


,raportfolioid,raportbenchmarkid,Portype,lab1,lab2,lab3,lab4,lab5,priority,postdt
0,1028,26592.0,FL,N/A,N/A,N/A,N/A,N/A,1,2004-07-01
1,16892,11049.0,AM,FUND,N/A,MF,AF,NG,1,2016-12-01
2,232413,15623.0,AM,FUND,N/A,MF,AF,VI,3,2017-03-01
3,152008,18779.0,AT,CM,N/A,N/A,N/A,N/A,4,2009-02-01
5,292128,23070.0,AM,FUND,N/A,MF,AF,VI,1,2017-10-01


In [117]:
#dedup
null_benchmarkids_filled.drop_duplicates(subset=['raportfolioid', 'postdt','raportbenchmarkid', 'priority'], keep='first', inplace=True)
print(null_benchmarkids_filled.shape)
null_benchmarkids_filled.head()

(430958, 10)


,raportfolioid,raportbenchmarkid,Portype,lab1,lab2,lab3,lab4,lab5,priority,postdt
4,242155,N/A,AM,FUND,N/A,MF,AF,NG,3,2016-12-01
10,127266,N/A,AA,SA,IA,N/A,N/A,N/A,4,2017-02-01
36,242155,N/A,AM,FUND,N/A,MF,AF,NG,2,2016-12-01
42,127266,N/A,AA,SA,IA,N/A,N/A,N/A,3,2017-01-01
68,242155,N/A,AM,FUND,N/A,MF,AF,NG,1,2016-12-01


In [118]:
min_null = null_benchmarkids_filled.groupby(['raportfolioid', 'postdt', 'raportbenchmarkid','Portype','lab1','lab2','lab3','lab4','lab5'])['priority'].min().reset_index()
min_port_df = port_df.groupby(['raportfolioid','postdt', 'raportbenchmarkid','Portype','lab1','lab2','lab3','lab4','lab5'])['priority'].min().reset_index()

In [169]:
f = {'raportbenchmarkid':['count']}
port_df_roll = min_port_df.groupby(['raportfolioid','postdt','Portype','priority','lab1','lab2','lab3','lab4','lab5']).agg(f).reset_index()
print(port_df_roll.shape)
port_df_roll.head()

(1436327, 10)


,raportfolioid,postdt,Portype,priority,lab1,lab2,lab3,lab4,lab5,raportbenchmarkid
,,,,,,,,,,count
0,1,1975-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6
1,1,1976-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6
2,1,1977-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6
3,1,1978-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6
4,1,1979-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6


In [170]:
port_df_roll.columns = port_df_roll.columns.droplevel(1)
port_df_roll.rename(columns={'raportbenchmarkid':'bmrk_cnt'}, inplace=True)
port_df_roll.head()

,raportfolioid,postdt,Portype,priority,lab1,lab2,lab3,lab4,lab5,bmrk_cnt
0,1,1975-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6
1,1,1976-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6
2,1,1977-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6
3,1,1978-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6
4,1,1979-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6


#### RA_BMRK_RTN

In [157]:
bmrk_df['priority'] = bmrk_df['rtnperiodtypcd'].map({'1MO':1, '3MO':2, '6MO':3, '12MO':4})
bmrk_df['postdt_new'] = bmrk_df['postdt'].apply(lambda x: x.replace(day=1))
bmrk_df.head()

KeyError: 'rtnperiodtypcd'

In [158]:
bmrk_df.drop(['rtnperiodtypcd','postdt'], axis=1, inplace=True)
bmrk_df.rename(columns={'postdt_new': 'postdt'}, inplace=True)
bmrk_df.head()

ValueError: labels ['rtnperiodtypcd'] not contained in axis

In [159]:
min_bmrk = bmrk_df.groupby(['raportfolioid', 'raportbenchmarkid','postdt'])['priority'].min().reset_index()
print(min_bmrk.shape)
min_bmrk.head()

(2405018, 4)


,raportfolioid,raportbenchmarkid,postdt,priority
0,1,1,1976-12-01,4
1,1,1,1977-12-01,4
2,1,1,1978-12-01,4
3,1,1,1979-12-01,4
4,1,1,1980-12-01,4


In [160]:
#dedup
min_bmrk = min_bmrk.drop_duplicates(subset=['raportfolioid', 'raportbenchmarkid', 'postdt','priority'], keep='first')
print(min_bmrk.shape)
min_bmrk.head()

(2405018, 4)


,raportfolioid,raportbenchmarkid,postdt,priority
0,1,1,1976-12-01,4
1,1,1,1977-12-01,4
2,1,1,1978-12-01,4
3,1,1,1979-12-01,4
4,1,1,1980-12-01,4


In [161]:
f = {'raportbenchmarkid':['count'], 'priority':['max']}
min_bmrk_roll = min_bmrk.groupby(['raportfolioid','postdt']).agg(f).reset_index()
print(min_bmrk_roll.shape)
min_bmrk_roll.head()

(1424576, 4)


,raportfolioid,postdt,raportbenchmarkid,priority
,,,count,max
0,1,1976-12-01,4,4
1,1,1977-12-01,4,4
2,1,1978-12-01,4,4
3,1,1979-12-01,4,4
4,1,1980-12-01,4,4


In [162]:
min_bmrk_roll.columns = min_bmrk_roll.columns.droplevel(1)
min_bmrk_roll.rename(columns={'raportbenchmarkid':'bmrk_rtn_cnt','priority':'bmrk_rtn_priority'}, inplace=True)
min_bmrk_roll.head()

,raportfolioid,postdt,bmrk_rtn_cnt,bmrk_rtn_priority
0,1,1976-12-01,4,4
1,1,1977-12-01,4,4
2,1,1978-12-01,4,4
3,1,1979-12-01,4,4
4,1,1980-12-01,4,4


In [171]:
print(port_df_roll.shape)
print(min_bmrk_roll.shape)
port_n_bmrk = port_df_roll.merge(min_bmrk_roll,left_on=['raportfolioid','postdt'], right_on=['raportfolioid','postdt'], how='left')
print(port_n_bmrk.shape)
port_n_bmrk.head()

(1436327, 10)
(1424576, 4)
(1436327, 12)


,raportfolioid,postdt,Portype,priority,lab1,lab2,lab3,lab4,lab5,bmrk_cnt,bmrk_rtn_cnt,bmrk_rtn_priority
0,1,1975-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,NaN,NaN
1,1,1976-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,4.0,4.0
2,1,1977-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,4.0,4.0
3,1,1978-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,4.0,4.0
4,1,1979-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,4.0,4.0


In [172]:
port_n_bmrk['HasHypo'] = port_n_bmrk['bmrk_cnt'] == port_n_bmrk['bmrk_rtn_cnt']


In [173]:
port_n_bmrk.head()

,raportfolioid,postdt,Portype,priority,lab1,lab2,lab3,lab4,lab5,bmrk_cnt,bmrk_rtn_cnt,bmrk_rtn_priority,HasHypo
0,1,1975-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,NaN,NaN,False
1,1,1976-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,4.0,4.0,False
2,1,1977-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,4.0,4.0,False
3,1,1978-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,4.0,4.0,False
4,1,1979-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,4.0,4.0,False


In [174]:
def has_relative_frequency(row):
    if row['HasHypo'] == True:
        return row[['priority', 'bmrk_rtn_priority']].max()
    if row['HasHypo'] == False:
        return row['priority']

In [176]:
port_n_bmrk['final_priority'] = port_n_bmrk.apply(lambda x: has_relative_frequency(x), axis=1)
port_n_bmrk.head()

,raportfolioid,postdt,Portype,priority,lab1,lab2,lab3,lab4,lab5,bmrk_cnt,bmrk_rtn_cnt,bmrk_rtn_priority,HasHypo,final_priority
0,1,1975-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,NaN,NaN,False,4.0
1,1,1976-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,4.0,4.0,False,4.0
2,1,1977-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,4.0,4.0,False,4.0
3,1,1978-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,4.0,4.0,False,4.0
4,1,1979-12-01,AA,4,MF,N/A,N/A,N/A,N/A,6,4.0,4.0,False,4.0


In [177]:
min_null.head()

,raportfolioid,postdt,raportbenchmarkid,Portype,lab1,lab2,lab3,lab4,lab5,priority
0,2,2004-09-01,N/A,AA,AG,N/A,N/A,N/A,N/A,1
1,43,2002-01-01,N/A,GM,N/A,N/A,N/A,N/A,N/A,1
2,43,2002-02-01,N/A,GM,N/A,N/A,N/A,N/A,N/A,1
3,43,2002-03-01,N/A,GM,N/A,N/A,N/A,N/A,N/A,1
4,43,2002-04-01,N/A,GM,N/A,N/A,N/A,N/A,N/A,1


In [181]:
port_n_bmrk.drop(['priority','bmrk_cnt','bmrk_rtn_cnt','bmrk_rtn_priority'], axis=1, inplace=True)
port_n_bmrk.head()

,raportfolioid,postdt,Portype,lab1,lab2,lab3,lab4,lab5,HasHypo,final_priority
0,1,1975-12-01,AA,MF,N/A,N/A,N/A,N/A,False,4.0
1,1,1976-12-01,AA,MF,N/A,N/A,N/A,N/A,False,4.0
2,1,1977-12-01,AA,MF,N/A,N/A,N/A,N/A,False,4.0
3,1,1978-12-01,AA,MF,N/A,N/A,N/A,N/A,False,4.0
4,1,1979-12-01,AA,MF,N/A,N/A,N/A,N/A,False,4.0


In [183]:
min_null['HasHypo'] = 'N/A'
min_null.drop(['raportbenchmarkid'], axis=1, inplace=True)
min_null.rename(columns={'priority':'final_priority'}, inplace=True)
min_null.head()

,raportfolioid,postdt,Portype,lab1,lab2,lab3,lab4,lab5,final_priority,HasHypo
0,2,2004-09-01,AA,AG,N/A,N/A,N/A,N/A,1,N/A
1,43,2002-01-01,GM,N/A,N/A,N/A,N/A,N/A,1,N/A
2,43,2002-02-01,GM,N/A,N/A,N/A,N/A,N/A,1,N/A
3,43,2002-03-01,GM,N/A,N/A,N/A,N/A,N/A,1,N/A
4,43,2002-04-01,GM,N/A,N/A,N/A,N/A,N/A,1,N/A


In [184]:
final_combo = port_n_bmrk.append(min_null,ignore_index=True)
print(final_combo.shape)
final_combo.head()

(1549019, 10)


,HasHypo,Portype,final_priority,lab1,lab2,lab3,lab4,lab5,postdt,raportfolioid
0,False,AA,4.0,MF,N/A,N/A,N/A,N/A,1975-12-01,1
1,False,AA,4.0,MF,N/A,N/A,N/A,N/A,1976-12-01,1
2,False,AA,4.0,MF,N/A,N/A,N/A,N/A,1977-12-01,1
3,False,AA,4.0,MF,N/A,N/A,N/A,N/A,1978-12-01,1
4,False,AA,4.0,MF,N/A,N/A,N/A,N/A,1979-12-01,1


In [185]:
df_summary = final_combo.groupby(['postdt','Portype','lab1','lab2','lab3','lab4','lab5','final_priority','HasHypo'])['raportfolioid'].count().reset_index()
df_summary.rename(columns={'raportfolioid':'raport_cnt'}, inplace=True)
print(df_summary.shape)
df_summary.head()

(25854, 10)


,postdt,Portype,lab1,lab2,lab3,lab4,lab5,final_priority,HasHypo,raport_cnt
0,1934-01-01,FL,N/A,N/A,N/A,N/A,N/A,1.0,N/A,1
1,1934-02-01,FL,N/A,N/A,N/A,N/A,N/A,1.0,N/A,1
2,1934-03-01,FL,N/A,N/A,N/A,N/A,N/A,1.0,N/A,1
3,1934-04-01,FL,N/A,N/A,N/A,N/A,N/A,1.0,N/A,1
4,1934-05-01,FL,N/A,N/A,N/A,N/A,N/A,1.0,N/A,1


In [186]:
#output the summarized version
df_summary.to_csv('RA_PORT_RTN_RLTV_wLabels_20181012.csv')